<a href="https://colab.research.google.com/github/heidingaway/datapeople/blob/main/opengov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from ckanapi import RemoteCKAN
import pandas as pd
import os
import zipfile

In [26]:
import yaml

In [36]:
TOKENcountry = "cac6fd9f-594a-4bcd-bf17-10295812d4c5"
rc = RemoteCKAN('https://open.canada.ca/data/en/', apikey=TOKENcountry)
result = rc.action.datastore_search(
    resource_id="bdb33e8c-53ef-4bae-9493-35f343191c02",
)
print(result['records'],5)

[{'_id': 1, 'GC_HIST_ID': '1000110', 'GC_ID': '10001', 'GC_NM_OFF_EN': 'the Republic of Fiji', 'GC_NM_OFF_FR': 'la République des Fidji', 'GC_NM_AB_EN': 'Fiji', 'GC_NM_AB_FR': 'Fidji', 'STAT_CD': '1', 'STAT_DESC_EN': 'ACTIVE', 'STAT_DESC_FR': 'ACTIVE', 'TYPE_CD': '1', 'TYPE_DESC_EN': 'Country', 'TYPE_DESC_FR': 'Pays', 'RECENT_IND': '1', 'EFF_DT': '1970-01-01T00:00:00', 'EFF_FN_DT': '2100-01-01T00:00:00', 'OBSERVATIONS_EN': 'NULL', 'OBSERVATIONS_FR': 'NULL', 'GC_PAR_ID': None, 'GC_JUR_ID': None, 'ISO_ALPHA_2_CD': 'FJ', 'ISO_ALPHA_3_CD': 'FJI', 'UN_ONU_CD': '242', 'MODIF_DT': '2023-11-28T00:00:00'}, {'_id': 2, 'GC_HIST_ID': '1000210', 'GC_ID': '10002', 'GC_NM_OFF_EN': 'South Georgia and the South Sandwich Islands', 'GC_NM_OFF_FR': 'Géorgie du Sud-et-les Îles Sandwich du Sud', 'GC_NM_AB_EN': 'South Georgia and the South Sandwich Islands', 'GC_NM_AB_FR': 'Géorgie du Sud-et-les Îles Sandwich du Sud', 'STAT_CD': '1', 'STAT_DESC_EN': 'ACTIVE', 'STAT_DESC_FR': 'ACTIVE', 'TYPE_CD': '2', 'TYPE_D

In [31]:
df = pd.DataFrame(result['records'])

# Add a new 'title' column using the content of 'GC_NM_AB_EN'
df['title'] = df['GC_NM_AB_EN']

In [32]:
df.to_csv('output.csv', index=False)
print(f"DataFrame saved to output.csv")

DataFrame saved to output.csv


In [33]:
# Create a directory to store the files
output_dir = 'markdown_files'
os.makedirs(output_dir, exist_ok=True)

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    # Create the file name (e.g., row_0.md, row_1.md, etc.)
    file_name = os.path.join(output_dir, f'row_{index}.md')

    # Convert the row to a dictionary
    row_dict = row.to_dict()

    # Relabel 'GC_NM_AB_EN:' to 'title:'
    if 'GC_NM_AB_EN:' in row_dict:
        row_dict['title:'] = row_dict.pop('GC_NM_AB_EN:')

    # Convert the dictionary to YAML format
    yaml_content = yaml.dump(row_dict, default_flow_style=False)

    # Write the YAML content to the file (with .md extension)
    with open(file_name, 'w') as f:
        f.write(yaml_content)

print(f"Files with YAML content (saved as .md) are in the '{output_dir}' directory.")

Files with YAML content (saved as .md) are in the 'markdown_files' directory.


In [34]:
# Create a zip file containing the Markdown files
zip_file_name = 'markdown_files.zip'
with zipfile.ZipFile(zip_file_name, 'w') as zipf:
    for root, _, files in os.walk(output_dir):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, output_dir))

print(f"Markdown files zipped to '{zip_file_name}'.")

Markdown files zipped to 'markdown_files.zip'.
